# eCommerce Product Recommendation - Part 2

# 1. Load Raw Data to Colab

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Laioffer/csv_files/RecSys_eComm/

/content/drive/MyDrive/Laioffer/csv_files/RecSys_eComm


In [ ]:
ls

aisles.csv       model_all_data.csv  order_products_prior.csv  orders.csv
departments.csv  model.png           order_products_train.csv  products.csv


In [ ]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products_prior = pd.read_csv('order_products_prior.csv')
order_products_train = pd.read_csv('order_products_train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

# 1. Construct Model Label

思考: 为什么不能直接使用`reordered`column中的数据作为model label？

reordered: 1 if this product has been ordered by this user in the past, 0 otherwise

- 买过1次 or 多次的，均为1. 从未购买过为0.
- 我们需要在1里，找出购买过2次 or 2次以上的客户。


思考：为什么我们需要使用both prior & train datasets




这个模型是为了预测未来有无复购，是classifier的问题，答案只有0/1。
只有prior和train里面reorder的label是1，然后label才是1。prior可能是now之前3个月或一年的时期的数据，prior里面的数据只能做feature，不能做label。但train里面的data只能用来做label，不能做feature。如果在prior里面label是0，那一定是没买过这个商品，所以在train里面的label一定是0；但只有两边（prior和train）里面都是1是，真正要predict的那个复购的label才会是1.


In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1363380,50,prior,1,3,9,NaN
1,3131103,50,prior,2,6,12,10.0
2,2197066,50,prior,3,1,13,9.0
3,3201640,50,prior,4,0,11,6.0
4,2756806,50,prior,5,4,14,11.0


In [ ]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1077,13176,1,1
1,1077,39922,2,1
2,1077,5258,3,1
3,1077,21137,4,1
4,1119,6046,1,1


In [ ]:
# （1）
train_details = order_products_train.merge(orders, on = 'order_id')
train_user_ids = set(orders[orders['eval_set'] == 'train']['user_id'])
train_unique_key = train_details['user_id'].astype('str') + '_' + train_details['product_id'].astype('str')

In [ ]:
train_details.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1077,13176,1,1,173934,train,11,6,9,10.0
1,1077,39922,2,1,173934,train,11,6,9,10.0
2,1077,5258,3,1,173934,train,11,6,9,10.0
3,1077,21137,4,1,173934,train,11,6,9,10.0
4,1119,6046,1,1,129386,train,7,1,14,17.0


In [ ]:
train_unique_key.head()

0    173934_13176
1    173934_39922
2     173934_5258
3    173934_21137
4     129386_6046
dtype: object

In [ ]:
# (2) 
prior_details = order_products_prior.merge(orders, on = 'order_id')

In [ ]:
#（3）
model_all_data = prior_details[prior_details.user_id.isin(train_user_ids)][['user_id','product_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']]
# 只保留/确保 所有的prior ID一定在train——ID里面
model_all_data = model_all_data.drop_duplicates(subset=['user_id', 'product_id']) #在model_all_data中，一个user对于一个product只应该有一行数据. #谨慎！！！！
model_all_data['unique_key'] = model_all_data['user_id'].astype('str') + '_' + model_all_data['product_id'].astype('str')

In [ ]:
#（4）
model_all_data['label'] = 0
model_all_data.loc[model_all_data.unique_key.isin(train_unique_key), 'label'] = 1  # 在prior和train里都买了。算复购。为1

<------------------ Tiffany(now) ------->
prior                            train 买了 yes （1） 或者 没买 No （0）

In [ ]:
model_all_data.head()

,user_id,product_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,unique_key,label
0,152610,30597,22,6,8,10.0,152610_30597,0
1,152610,15221,22,6,8,10.0,152610_15221,0
2,152610,43772,22,6,8,10.0,152610_43772,0
3,152610,37886,22,6,8,10.0,152610_37886,0
4,152610,37215,22,6,8,10.0,152610_37215,0


# 2. Construct Model Features

思考：为什么我们需要单独使用prior dataset

2.1. Feature Group 1: user-product activity features （用户买了什么东西）

In [ ]:
#注意命名技巧！！！！！！
user_product_features = ['user_product__total_orders',
                         'user_product__add_to_cart_order_mean',
                         'user_product__reordered_mean',
                         'user_product__most_dow',
                         'user_product__most_hod']

In [ ]:
prior_details.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,12,30597,1,1,152610,prior,22,6,8,10.0
1,12,15221,2,1,152610,prior,22,6,8,10.0
2,12,43772,3,1,152610,prior,22,6,8,10.0
3,12,37886,4,1,152610,prior,22,6,8,10.0
4,12,37215,5,0,152610,prior,22,6,8,10.0


In [ ]:
# 注意: 这个block的程序执行时间约为2mins
df_user_product_features = (prior_details.groupby(['product_id','user_id'],as_index=False) 
                                           .agg(OrderedDict(
                                                   [('order_id','count'), #该用户对该商品的购买次数
                                                    ('add_to_cart_order','mean'), # 该用户对该商品的加入购物车顺序 - 均值
                                                    ('reordered', 'mean'), #该用户对该商品的复购率
                                                    ('order_dow', (lambda x: x.mode()[0])), #该用户购买该商品，最多的dow
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该用户购买该商品，最多的 hod
                                                    ])))
#这里建OrderDict要符合上面user_product_features的顺序
df_user_product_features.columns = ['product_id', 'user_id'] + user_product_features

In [ ]:
model_all_data = model_all_data.merge(df_user_product_features, on = ['user_id', 'product_id'])

2.2. Feature Group 2: product features （间接features，如何描述这些products）

product features分为两种，一种是根据所有用户的行为总结出的product features, 另一种是根据product本身信息得到的features。

In [ ]:
product_features = ['product__total_orders',
                     'product__add_to_cart_order_mean',
                     'product__total_users',
                     'product__reordered_mean',
                     'product__most_dow',
                     'product__most_hod',
                     'product__days_since_prior_order_mean'
                     ]

In [ ]:
df_product_features = (prior_details.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), # 该产品被所有用户购买的次数
                                                    ('add_to_cart_order','mean'), #该产品被所有用户添加到购物车的顺序 - 平均值
                                                    ('user_id', 'nunique'), # 所有购买该产品的人数
                                                    ('reordered', 'mean'), #该产品被所有用户复购的频率 (也可以加总和，方差)
                                                    ('order_dow', (lambda x: x.mode()[0])), #该产品被购买最多的dow #
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该产品被购买最多的dow
                                                    ('days_since_prior_order', 'mean') #该商品距离上次被购买的均值
                                                    ])))
df_product_features.columns = ['product_id'] + product_features

In [ ]:
model_all_data = model_all_data.merge(df_product_features, on = ['product_id'])

In [ ]:
model_all_data = model_all_data.merge(products[['product_id','aisle_id', 'department_id']], 
                                      on = ['product_id'])

In [ ]:
model_all_data.rename(columns={'aisle_id': 'product__aisle_id', 'department_id': 'product__department_id'}, inplace=True)

2.3. Feature Group 3: user features

In [ ]:
user_features = ['user__order_count',
                  'user__product_count',
                  'user__days_since_prior_order_mean',
                  'user__reordered_mean',
                  'user__most_dow',
                  'user__most_hod',
                  ]

In [ ]:
df_user_features = (prior_details.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'), # 该用户的总订单数
                                                    ('product_id','count'), #该用户购买的全部商品总数
                                                    ('days_since_prior_order','mean'), #该用户订单平均间隔时间
                                                    ('reordered', 'mean'), #该用户的总复购率
                                                    ('order_dow', (lambda x: x.mode()[0])), #该用户下单最多的dow
                                                    ('order_hour_of_day', (lambda x: x.mode()[0])), #该用户下单最多的dow
                                                    ])))
df_user_features.columns = ['user_id'] + user_features

In [ ]:
model_all_data = model_all_data.merge(df_user_features, on = ['user_id'])

In [ ]:
model_all_data['label'].value_counts()

0    297467
1     32339
Name: label, dtype: int64

In [ ]:
model_all_data.shape

(329806, 28)

# 3. Save feature data to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# Save the model_all_data dataframe as csv file to the file path we selected above.
model_all_data.to_csv('model_all_data.csv', index=False)

* 面试时常到此结束了
* 越早讨论Models，你肯定挂得越快。